### Content Based Recommendations

In the previous notebook, you were introduced to a way to make recommendations using collaborative filtering.  However, using this technique there are a large number of users who were left without any recommendations at all.  Other users were left with fewer than the ten recommendations that were set up by our function to retrieve...

In order to help these users out, let's try another technique **content based** recommendations.  Let's start off where we were in the previous notebook.

In [1]:
!pip install progressbar

  Running setup.py bdist_wheel for progressbar ... done
  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built progressbar


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import ast
from collections import defaultdict
from IPython.display import HTML
import progressbar
import tests as t
import pickle


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [3]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Documentary,Adventure,Animation,...,War,Romance,Musical,Film-Noir,Sport,Mystery,Western,Horror,Comedy,Music
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,114508,8,1381006850,2013-10-05 21:00:50
1,2,208092,5,1586466072,2020-04-09 21:01:12
2,2,358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17


In [3]:
# Do not change this code
# This loads our solution dictionary (key: user_id and value: an array of recommended movie titles)
all_recs = {}
with open("all_recs.txt", newline="") as csvfile:
    data = csv.reader(csvfile, delimiter=",")
    for row in data:
        val = ast.literal_eval(row[1])
        all_recs[int(row[0])] = val

In [6]:
all_recs

{2: ['The Big Sleep (1946)',
  'A Streetcar Named Desire (1951)',
  "Ascenseur pour l'échafaud (1958)",
  'À bout de souffle (1960)',
  'Le petit soldat (1963)',
  'Vivre sa vie: Film en douze tableaux (1962)',
  "2 ou 3 choses que je sais d'elle (1967)",
  'The Way We Were (1973)',
  'A Woman Under the Influence (1974)',
  'Barry Lyndon (1975)',
  'Apocalypse Now (1979)',
  'El-haimoune (1984)',
  'Paris, Texas (1984)',
  'Au revoir les enfants (1987)',
  'Akira (1988)',
  'Goodfellas (1990)',
  'Trois couleurs: Bleu (1993)',
  'Legends of the Fall (1994)',
  'Trois couleurs: Blanc (1994)',
  'Faa yeung nin wa (2000)',
  'Chun gwong cha sit (1997)',
  'Elizabeth (1998)',
  'La fille sur le pont (1999)',
  'Lilja 4-ever (2002)',
  'Cidade de Deus (2002)',
  'Brokeback Mountain (2005)',
  'Elizabeth: The Golden Age (2007)',
  'The Tree of Life (2011)',
  'Papurika (2006)',
  'Doubt (2008)',
  '4 luni, 3 saptamâni si 2 zile (2007)',
  'Incendies (2010)',
  'Kynodontas (2009)',
  'Prisone

### Datasets

From the above, you now have access to three important items that you will be using throughout the rest of this notebook.  

`a.` **movies** - a dataframe of all of the movies in the dataset along with other content related information about the movies (genre and date)


`b.` **reviews** - this was the main dataframe used before for collaborative filtering, as it contains all of the interactions between users and movies.


`c.` **all_recs** - a dictionary where each key is a user, and the value is a list of movie recommendations based on collaborative filtering

For the individuals in **all_recs** who did recieve 10 recommendations using collaborative filtering, we don't really need to worry about them.  However, there were a number of individuals in our dataset who did not receive any recommendations.

-----

`1.` To begin, let's start with finding all of the users in our dataset who didn't get all 10 ratings we would have liked them to have using collaborative filtering.  

In [4]:
users_with_all_recs = []
# Convert `all_recs` dictionary into array of users with all recommendations


for user, movie_list in all_recs.items():
    if len(movie_list) > 9:
        users_with_all_recs.append(user)

print("There are {} users with all reccomendations from collaborative filtering.".format(len(users_with_all_recs)))

users = np.unique(reviews['user_id'])
users_who_need_recs = np.setdiff1d(users, users_with_all_recs)
        
print("There are {} users who still need recommendations.".format(len(users_who_need_recs)))
print("This means that only {}% of users received all 10 of their recommendations using collaborative filtering".format(round(len(users_with_all_recs)/len(np.unique(reviews['user_id'])), 4)*100))

There are 3494 users with all reccomendations from collaborative filtering.
There are 4528 users who still need recommendations.
This means that only 43.56% of users received all 10 of their recommendations using collaborative filtering


In [11]:
# Some test here might be nice
assert len(users_who_need_recs) == 4528
print("That's right there were still another 4528 users who needed recommendations when we only used collaborative filtering!")

That's right there were still another 4528 users who needed recommendations when we only used collaborative filtering!


### Content Based Recommendations

You will be doing a bit of a mix of content and collaborative filtering to make recommendations for the users this time.  This will allow you to obtain recommendations in many cases where we didn't make recommendations earlier.     

`2.` Before finding recommendations, rank the user's ratings from highest ratings to lowest ratings. You will move through the movies in this order looking for other similar movies.

In [5]:
# create a dataframe similar to reviews, but ranked by rating for each user
ranked_reviews = reviews.sort_values(by = ['user_id', 'rating'], ascending = False)

In [14]:
ranked_reviews.head()

,user_id,movie_id,rating,timestamp,date
99993,8022,25316,9,1587611714,2020-04-23 03:15:14
99994,8022,33467,9,1586045702,2020-04-05 00:15:02
99995,8022,34583,9,1582589387,2020-02-25 00:09:47
99996,8022,40746,9,1585954942,2020-04-03 23:02:22
99997,8022,41959,9,1586569384,2020-04-11 01:43:04


### Similarities

In the collaborative filtering sections, you became quite familiar with different methods of determining the similarity (or distance) of two users.  We can perform similarities based on content in much the same way.  

In many cases, it turns out that one of the fastest ways we can find out how similar items are to one another (when our matrix isn't totally sparse like it was in the earlier section) is by simply using matrix multiplication.  If you are not familiar with this, an explanation is available [here by 3blue1brown](https://www.youtube.com/watch?v=LyGKycYT2v0) and another quick explanation is provided [on the post here](https://math.stackexchange.com/questions/689022/how-does-the-dot-product-determine-similarity).

For us to pull out a matrix that describes the movies in our dataframe in terms of content, we might just use the indicator variables related to **year** and **genre** for our movies.  

Then we can obtain a matrix of how similar movies are to one another by taking the dot product of this matrix with itself.  Notice in the below that the dot product where our 1 values overlap gives a value of 2 indicating higher similarity.  In the second dot product, the 1 values don't match up.  This leads to a dot product of 0 indicating lower similarity.

<img src="images/dotprod1.png" alt="Dot Product" height="500" width="500">

We can perform the dot product on a matrix of movies with content characteristics to provide a movie by movie matrix where each cell is an indication of how similar two movies are to one another.  In the below image, you can see that movies 1 and 8 are most similar, movies 2 and 8 are most similar and movies 3 and 9 are most similar for this subset of the data.  The diagonal elements of the matrix will contain the similarity of a movie with itself, which will be the largest possible similarity (which will also be the number of 1's in the movie row within the orginal movie content matrix.

<img src="images/moviemat.png" alt="Dot Product" height="500" width="500">


`3.` Create a numpy array that is a matrix of indicator variables related to year (by century) and movie genres by movie.  Perform the dot prodoct of this matrix with itself (transposed) to obtain a similarity matrix of each movie with every other movie.  The final matrix should be 31245 x 31245.

In [6]:
# Subset so movie_content is only using the dummy variables for each genre and the 3 century based year dummy columns
movie_content = np.array(movies.iloc[:,4:])
movie_content[1:4,1:4]

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 0]])

In [7]:
# Take the dot product to obtain a movie x movie matrix of similarities
dot_prod_movies = movie_content.dot(np.transpose(movie_content))

In [25]:
# create checks for the dot product matrix
assert dot_prod_movies.shape[0] == 35479
assert dot_prod_movies.shape[1] == 35479
assert dot_prod_movies[0, 0] == np.max(dot_prod_movies[0])
print("Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!")

Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!


In [26]:
dot_prod_movies[:4,:4]

array([[3, 3, 3, 1],
       [3, 3, 3, 1],
       [3, 3, 3, 1],
       [1, 1, 1, 1]])

### For Each User...


Now that you have a matrix where each user has their ratings ordered.  You also have a second matrix where movies are each axis, and the matrix entries are larger where the two movies are more similar and smaller where the two movies are dissimilar.  This matrix is a measure of content similarity. Therefore, it is time to get to the fun part.

For each user, we will perform the following:

    i. For each movie, find the movies that are most similar that the user hasn't seen.

    ii. Continue through the available, rated movies until 10 recommendations or until there are no additional movies.

As a final note, you may need to adjust the criteria for 'most similar' to obtain 10 recommendations.  As a first pass, I used only movies with the highest possible similarity to one another as similar enough to add as a recommendation.

`3.` In the below cell, complete each of the functions needed for making content based recommendations.

In [29]:
movie_id = 25316
movie_idx = np.where(movies['movie_id'] == movie_id)[0][0]
movie_idx

442

In [9]:
recs = defaultdict(set)
recs

defaultdict(set, {})

In [10]:
def find_similar_movies(movie_id):
    '''
    INPUT
    movie_id - a movie_id 
    OUTPUT
    similar_movies - an array of the most similar movies by title
    '''
    # Implement your code here
    movie_idx = np.where(movies['movie_id'] == movie_id)[0][0]
    
    sim_movies_idx = np.where(dot_prod_movies[movie_idx] == np.max(dot_prod_movies[movie_idx]))[0]
    
    similar_movies = np.array(movies.iloc[sim_movies_idx,]['movie'])
                                                                                          
    return similar_movies
    
    
def get_movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    # Implement your code here
    movie_lst = list(movies[movies['movie_id'].isin(movie_ids)]['movie'])
    return movie_lst

def make_recs():
    '''
    INPUT
    None
    OUTPUT
    recs - a dictionary with keys of the user and values of the recommendations
    '''
    # Implement your code here
        # Create dictionary to return with users and ratings
    recs = defaultdict(set)
    # How many users for progress bar
    n_users = len(users)

    
    # Create the progressbar
    cnter = 0
    bar = progressbar.ProgressBar(maxval=n_users+1, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    # For each user
    for user in users:
        
        # Update the progress bar
        cnter+=1 
        bar.update(cnter)

        # Pull only the reviews the user has seen
        reviews_temp = ranked_reviews[ranked_reviews['user_id'] == user]
        movies_temp = np.array(reviews_temp['movie_id'])
        movie_names = np.array(get_movie_names(movies_temp))

        # Look at each of the movies (highest ranked first), 
        # pull the movies the user hasn't seen that are most similar
        # These will be the recommendations - continue until 10 recs 
        # or you have depleted the movie list for the user
        for movie in movies_temp:
            rec_movies = find_similar_movies(movie)
            temp_recs = np.setdiff1d(rec_movies, movie_names)
            recs[user].update(temp_recs)

            # If there are more than 
            if len(recs[user]) > 9:
                break

    bar.finish()
                              
                              
    return recs

In [11]:
recs = make_recs()

[========================================================================] 100%


### How Did We Do?

Now that you have made the recommendations, how did we do in providing everyone with a set of recommendations?

`4.` Use the cells below to see how many individuals you were able to make recommendations for, as well as explore characteristics about individuals who you were not able to make recommendations for.  

In [12]:
# Explore recommendations for
# users without all 10 recommendations and store it in `users_without_all_recs=[]`
# users with all 10 recommendations and store it in users_with_all_recs=[]
# users with no recommendations at all and store it in no_recs=[]
users_without_all_recs = []
users_with_all_recs = []
no_recs = []
for user, movie_recs in recs.items():
    if len(movie_recs) < 10:
        users_without_all_recs.append(user)
    if len(movie_recs) > 9:
        users_with_all_recs.append(user)
    if len(movie_recs) == 0:
        no_recs.append(user)


In [13]:
# Some characteristics of my content based recommendations
print("There were {} users without all 10 recommendations we would have liked to have.".format(len(users_without_all_recs)))
print("There were {} users with all 10 recommendations we would like them to have.".format(len(users_with_all_recs)))
print("There were {} users with no recommendations at all!".format(len(no_recs)))

There were 322 users without all 10 recommendations we would have liked to have.
There were 7700 users with all 10 recommendations we would like them to have.
There were 77 users with no recommendations at all!


In [14]:
# Closer look at individual user characteristics
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    # Implement your code here
    movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].isnull() == False].index.values

    return movies

movies_watched(189)

array([1392214, 1454468])

In [21]:
cnter = 0
print("Some of the movie lists for users without any recommendations include:")
for user_id in no_recs:
    print(user_id)
    print(get_movie_names(movies_watched(user_id)))
    cnter+=1
    if cnter > 10:
        break

Some of the movie lists for users without any recommendations include:
26
['Les Misérables (2012)']
48
['Wallace & Gromit in The Wrong Trousers (1993)']
330
['Resident Evil: Vendetta (2017)']
410
['Loving Vincent (2017)']
634
['Maleficent (2014)']
678
['Coco (2017)']
741
['Papillon (2017)']
820
['The Ballad of Buster Scruggs (2018)']
897
['Coco (2017)']
907
['Wonder Woman (2017)']
1077
['Wonder Woman (2017)']


### Now What?  

Well, if you were really strict with your criteria for how similar two movies (like I was initially), then you still have some users that don't have all 10 recommendations (and a small group of users who have no recommendations at all). 

As stated earlier, recommendation engines are a bit of an **art** and a **science**.  There are a number of things we still could look into - how do our collaborative filtering and content based recommendations compare to one another? How could we incorporate user input along with collaborative filtering and/or content based recommendations to improve any of our recommendations?  How can we truly gain recommendations for every user?

`5.` In this last step feel free to explore any last ideas you have with the recommendation techniques we have looked at so far.  You might choose to make the final needed recommendations using the first technique with just top ranked movies.  You might also loosen up the strictness in the similarity needed between movies.  Be creative and share your insights with your classmates!

In [ ]:
# Cells for exploring